In [1]:
from src.preprocessing import preprocess
from src.voice_conversion import voice_convert
from src.automatic_speech_recognition import asr
from src.speaker_verification import process_data_to_embeddings, compute_metrics_and_similarity
from src.utils import CODEBASE_DIR

/om2/user/azain/anaconda/envs/voice-anon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1039)
           2	LOAD_FAST(arg=0, lineno=1042)
           4	LOAD_CONST(arg=1, lineno=1042)
           6	BINARY_SUBSCR(arg=None, lineno=1042)
           8	LOAD_FAST(arg=0, lineno=1042)
          10	LOAD_CONST(arg=2, lineno=1042)
          12	BINARY_SUBSCR(arg=None, lineno=1042)
          14	COMPARE_OP(arg=4, lineno=1042)
          16	LOAD_FAST(arg=0, lineno=1042)
          18	LOAD_CONST(arg=1, lineno=1042)
          20	BINARY_SUBSCR(arg=None, lineno=1042)
          22	LOAD_FAST(arg=0, lineno=1042)
          24	LOAD_CONST(arg=3, lineno=1042)
          26	BINARY_SUBSCR(arg=None, lineno=1042)
          28	COMPARE_OP(arg=5, lineno=1042)
          30	BINARY_AND(arg=None, lineno=1042)
          32	RETURN_VALUE(arg=None, lineno=1042)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:state.pc_initial: State(pc_initial=0 nstack_

/om2/user/azain/anaconda/envs/voice-anon/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Loading checkpoint...
INFO:root:Loaded checkpoint '/net/vast-storage/scratch/vast/gablab/azain/code/voice_anonymization/src/../../FreeVC/checkpoints/freevc.pth' (iteration 1372)
Loading WavLM for content...
INFO:wavlm.WavLM:WavLM Config: {'extractor_mode': 'layer_norm', 'encoder_layers': 24, 'encoder_embed_dim': 1024, 'encoder_ffn_embed_dim': 4096, 'encoder_attention_heads': 16, 'activation_fn': 'gelu', 'layer_norm_first': True, 'conv_feature_layers': '[(512,10,5)] + [(512,3,2)] * 4 + [(512,2,2)] * 2', 'conv_bias': False, 'feature_grad_mult': 1.0, 'normalize': True, 'dropout': 0.0, 'attention_dropout': 0.0, 'activation_dropout': 0.0, 'encoder_layerdrop': 0.0, 'dropout_input': 0.0, 'dropout_features': 0.0, 'mask_length': 10, 'mask_prob': 0.8, 'mask_selection': 'static', 'mask_other': 0.0, 'no_mask_overlap': False, 'mask_min_space': 1, 'mask_channel_length': 10, 'mask_channel_prob': 0.0, 'mask_channel_selection': 'static', 'mask_channel_other': 0.0, 'no_mask_channel_overlap': False, 'mas

In [2]:
raw_data_path = f'{CODEBASE_DIR}/data/raw/LibriTTS/dev-clean'
processed_data_path = f'{CODEBASE_DIR}/data/processed/LibriTTS-dev-clean-16khz-mono-loudnorm'
transcript_path_pattern = "{base_name}.original.txt"
speaker_id_pattern = r"(\d+)_"
target_sample_rate = 16000
num_samples = 100
dataset_name = f"azain/LibriTTS-dev-clean-16khz-mono-loudnorm-100-random-samples-2024-04-18-17-34-39"
split = 'dev'
file_extensions = ['.wav']
target_index = 0
asr_model_id = "openai/whisper-tiny.en"

In [4]:
dataset = preprocess(
    raw_data_path, processed_data_path, transcript_path_pattern, speaker_id_pattern, 
    target_sample_rate, num_samples, file_extensions
    )
converted_dataset = voice_convert(dataset, target_index)
dataset_after_asr = asr(asr_model_id, dataset, split)
converted_dataset_after_asr = asr(asr_model_id, converted_dataset, split)
dataset_with_asr_and_embeddings = process_data_to_embeddings(dataset_after_asr)
converted_dataset_with_asr_and_embeddings = process_data_to_embeddings(converted_dataset_after_asr)
similarities, eer_score, wer_score = compute_metrics_and_similarity(dataset_with_asr_and_embeddings)
converted_similarties, converted_eer_score, converted_wer_score = compute_metrics_and_similarity(converted_dataset_with_asr_and_embeddings)

 44%|████▎     | 7574/17400 [00:00<00:00, 10839.92it/s]

Skipping normalization due to error: Audio must have length greater than the block size.
didnt normalize loudness for 2428_83705_000000_000000.wav


 66%|██████▌   | 11437/17400 [00:01<00:00, 11258.56it/s]

Skipping normalization due to error: Audio must have length greater than the block size.
didnt normalize loudness for 3853_163249_000146_000000.wav
Skipping normalization due to error: Audio must have length greater than the block size.
didnt normalize loudness for 6313_66125_000034_000000.wav


 79%|███████▉  | 13824/17400 [00:01<00:00, 10560.44it/s]

...

computing metrics and similarity similarity for Dataset({
    features: ['audio', 'transcript', 'speaker_id', 'asr_transcription', 'embeddings'],
    num_rows: 100
})


100%|██████████| 100/100 [00:13<00:00,  7.40it/s]


done computing metrics and pairwise similarity
computing metrics and similarity similarity for Dataset({
    features: ['audio', 'transcript', 'speaker_id', 'asr_transcription', 'embeddings'],
    num_rows: 100
})


100%|██████████| 100/100 [00:12<00:00,  7.84it/s]


done computing metrics and pairwise similarity


In [5]:
dataset_with_asr_and_embeddings

Dataset({
    features: ['audio', 'transcript', 'speaker_id', 'asr_transcription', 'embeddings'],
    num_rows: 100
})

In [6]:
dataset_with_asr_and_embeddings[0]

{'audio': {'path': None,
  'array': array([-0.00137329, -0.00134277, -0.00048828, ..., -0.0010376 ,
          0.00134277,  0.0007019 ]),
  'sampling_rate': 16000},
 'transcript': 'The mountaineer quickly formed a loop in one end of the rope, making it large enough to permit of its slipping over the shoulders of a man.',
 'speaker_id': '6313',
 'asr_transcription': {'text': ' the mountaineer quickly formed a loop in one end of the rope, making it large enough to permit of it slipping over the shoulders of a man.'},
 'embeddings': [-15.214618682861328,
  -5.800723075866699,
  1.344504475593567,
  -5.173941135406494,
  -6.602982044219971,
  17.321691513061523,
  17.280759811401367,
  26.46316146850586,
  -0.19831949472427368,
  -20.185012817382812,
  -28.183467864990234,
  -4.2336931228637695,
  9.371749877929688,
  -15.088933944702148,
  11.905731201171875,
  -6.78711462020874,
  -3.5590527057647705,
  -12.564376831054688,
  0.21996338665485382,
  -4.892398357391357,
  -2.22225284576416,

In [8]:
converted_dataset_with_asr_and_embeddings[0]

{'audio': {'path': None,
  'array': array([-1.52587891e-04,  2.74658203e-04,  2.86865234e-03, ...,
          9.76562500e-04,  1.46484375e-03,  6.10351562e-05]),
  'sampling_rate': 16000},
 'transcript': 'The mountaineer quickly formed a loop in one end of the rope, making it large enough to permit of its slipping over the shoulders of a man.',
 'speaker_id': '6313',
 'asr_transcription': {'text': ' the matinee are quickly formed of loop in one end of the rope, making it large enough to permit of its slipping over the shoulders of a man.'},
 'embeddings': [-18.25465965270996,
  -10.41771411895752,
  12.926801681518555,
  32.50221252441406,
  1.5055961608886719,
  18.315080642700195,
  8.612079620361328,
  18.72712516784668,
  -6.5341315269470215,
  -24.514925003051758,
  -25.768245697021484,
  13.07422161102295,
  27.211963653564453,
  -12.965782165527344,
  24.19794464111328,
  -0.29035431146621704,
  15.153406143188477,
  -19.999067306518555,
  13.285738945007324,
  -11.08647441864013

In [9]:
similarities, eer_score, wer_score

(Dataset({
     features: ['speaker_id1', 'speaker_id2', 'embedding1', 'embedding2', 'similarity_score'],
     num_rows: 4950
 }),
 0.056532686576247215,
 0.18932874500751495)

In [11]:
similarities[0]

{'speaker_id1': '6313',
 'speaker_id2': '2412',
 'embedding1': [-15.214618682861328,
  -5.800723075866699,
  1.344504475593567,
  -5.173941135406494,
  -6.602982044219971,
  17.321691513061523,
  17.280759811401367,
  26.46316146850586,
  -0.19831949472427368,
  -20.185012817382812,
  -28.183467864990234,
  -4.2336931228637695,
  9.371749877929688,
  -15.088933944702148,
  11.905731201171875,
  -6.78711462020874,
  -3.5590527057647705,
  -12.564376831054688,
  0.21996338665485382,
  -4.892398357391357,
  -2.22225284576416,
  -16.194238662719727,
  -19.011676788330078,
  17.45590591430664,
  -12.875974655151367,
  -5.862804889678955,
  -3.4907660484313965,
  15.980929374694824,
  9.094974517822266,
  -1.6013855934143066,
  -17.345252990722656,
  -18.03551483154297,
  3.450981855392456,
  -15.870384216308594,
  25.897253036499023,
  -22.915273666381836,
  3.08756685256958,
  24.622058868408203,
  10.33957576751709,
  8.606734275817871,
  -23.4212646484375,
  -19.228721618652344,
  21.190

In [10]:
converted_similarties, converted_eer_score, converted_wer_score

(Dataset({
     features: ['speaker_id1', 'speaker_id2', 'embedding1', 'embedding2', 'similarity_score'],
     num_rows: 4950
 }),
 0.4180276095867157,
 0.223178431391716)

In [12]:
converted_similarties[0]

{'speaker_id1': '6313',
 'speaker_id2': '2412',
 'embedding1': [-18.25465965270996,
  -10.41771411895752,
  12.926801681518555,
  32.50221252441406,
  1.5055961608886719,
  18.315080642700195,
  8.612079620361328,
  18.72712516784668,
  -6.5341315269470215,
  -24.514925003051758,
  -25.768245697021484,
  13.07422161102295,
  27.211963653564453,
  -12.965782165527344,
  24.19794464111328,
  -0.29035431146621704,
  15.153406143188477,
  -19.999067306518555,
  13.285738945007324,
  -11.086474418640137,
  -2.7500452995300293,
  -27.55145263671875,
  22.4255313873291,
  5.605979919433594,
  -23.871089935302734,
  -22.251352310180664,
  -25.742258071899414,
  0.7887747287750244,
  19.696062088012695,
  -19.13656234741211,
  -15.277641296386719,
  -17.940322875976562,
  2.571335554122925,
  -5.140568733215332,
  18.173994064331055,
  -19.937118530273438,
  18.65091323852539,
  16.10129165649414,
  20.776060104370117,
  -14.353499412536621,
  -35.411197662353516,
  -9.566215515136719,
  29.806